In [5]:
import ee
import geemap
import math
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
Map = geemap.Map(center=[40, -100], zoom=4)

In [7]:
# Define study area.
TIGER = ee.FeatureCollection('TIGER/2018/Counties')
region = ee.Feature(TIGER \
    .filter(ee.Filter.eq('STATEFP', '17')) \
    .filter(ee.Filter.eq('NAME', 'McLean')) \
    .first())
geometry = region.geometry()
Map.centerObject(region)
Map.addLayer(region, {
    'color': 'red'
}, 'McLean County')


In [8]:
# Import Landsat imagery.
landsat7 = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2')
landsat8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')

In [9]:
# Functions to rename Landsat 7 and 8 images.
def renameL7(img):
    return img.rename(['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1',
        'SWIR2', 'TEMP1', 'ATMOS_OPACITY', 'QA_CLOUD',
        'ATRAN', 'CDIST',
        'DRAD', 'EMIS', 'EMSD', 'QA', 'TRAD', 'URAD',
        'QA_PIXEL',
        'QA_RADSAT'
    ])

In [10]:
def renameL8(img):
    return img.rename(['AEROS', 'BLUE', 'GREEN', 'RED', 'NIR',
        'SWIR1',
        'SWIR2', 'TEMP1', 'QA_AEROSOL', 'ATRAN', 'CDIST',
        'DRAD', 'EMIS',
        'EMSD', 'QA', 'TRAD', 'URAD', 'QA_PIXEL', 'QA_RADSAT'
    ])

In [11]:
# Functions to mask out clouds, shadows, and other unwanted features.
def addMask(img):
    # Bit 0: Fill
    # Bit 1: Dilated Cloud
    # Bit 2: Cirrus (high confidence) (L8) or unused (L7)
    # Bit 3: Cloud
    # Bit 4: Cloud Shadow
    # Bit 5: Snow
    # Bit 6: Clear
    #        0: Cloud or Dilated Cloud bits are set
    #        1: Cloud and Dilated Cloud bits are not set
    # Bit 7: Water
    clear = img.select('QA_PIXEL').bitwiseAnd(64).neq(0)
    clear = clear.updateMask(clear).rename(['pxqa_clear'])

    water = img.select('QA_PIXEL').bitwiseAnd(128).neq(0)
    water = water.updateMask(water).rename(['pxqa_water'])

    cloud_shadow = img.select('QA_PIXEL').bitwiseAnd(16).neq(0)
    cloud_shadow = cloud_shadow.updateMask(cloud_shadow).rename([
        'pxqa_cloudshadow'
    ])

    snow = img.select('QA_PIXEL').bitwiseAnd(32).neq(0)
    snow = snow.updateMask(snow).rename(['pxqa_snow'])

    masks = ee.Image.cat([
        clear, water, cloud_shadow, snow
    ])

    return img.addBands(masks)

In [12]:
def maskQAClear(img):
    return img.updateMask(img.select('pxqa_clear'))

In [13]:
# Function to add GCVI as a band.
def addVIs(img):
    gcvi = img.expression('(nir / green) - 1', {
      'nir': img.select('NIR'),
      'green': img.select('GREEN')
    }).select([0], ['GCVI'])
    
    # gcvi = img.expression('(nir - red) / (nir + red)', {
    #   'nir': img.select('NIR'),
    #   'red': img.select('RED')
    # }).select([0], ['GCVI'])

    return ee.Image.cat([img, gcvi])

In [14]:
# Función para aplicar los factores de escala a las bandas
def apply_scale_factors_L8(image):
    optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    return image.addBands(optical_bands, overwrite=True)

In [15]:
def apply_scale_factors_L7(image):
    optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    return image.addBands(optical_bands, overwrite=True)

In [16]:
# Define study time period.
start_date = '2020-05-01'
end_date = '2020-11-30'

# Pull Landsat 7 and 8 imagery over the study area between start and end dates.
landsat7coll = landsat7 \
    .filterBounds(geometry) \
    .filterDate(start_date, end_date) \
    .map(apply_scale_factors_L7) \
    .map(renameL7)

landsat8coll = landsat8 \
    .filterDate(start_date, end_date) \
    .filterBounds(geometry) \
    .map(apply_scale_factors_L8) \
    .map(renameL8)

# Merge Landsat 7 and 8 collections.
landsat = landsat7coll.merge(landsat8coll) \
    .sort('system:time_start')

# Merge Landsat 7 and 8 collections.
# landsat = landsat8coll

# Mask out non-clear pixels, add VIs and time variables.
landsat = landsat.map(addMask) \
    .map(maskQAClear) \
    .map(addVIs)



In [17]:
# Visualize GCVI time series at one location.
lat =  40.579804398254005
long = -88.81417685576481

point = ee.Geometry.Point([long,lat
])

In [18]:
# Function that adds time band to an image.
def addTimeUnit(image, refdate):
    date = image.date()

    dyear = date.difference(refdate, 'year')
    t = image.select(0).multiply(0).add(dyear).select([0], ['t']) \
        .float()

    imageplus = image.addBands(t)

    return imageplus


# Function that adds harmonic basis to an image.
def addHarmonics(image, omega, refdate):
    image = addTimeUnit(image, refdate)
    timeRadians = image.select('t').multiply(2 * math.pi * omega)
    timeRadians2 = image.select('t').multiply(4 * math.pi *
    omega)

    return image \
        .addBands(timeRadians.cos().rename('cos')) \
        .addBands(timeRadians.sin().rename('sin')) \
        .addBands(timeRadians2.cos().rename('cos2')) \
        .addBands(timeRadians2.sin().rename('sin2')) \
        .addBands(timeRadians.divide(timeRadians) \
            .rename('constant'))


# Apply addHarmonics to Landsat image collection.
omega = 1
# Definir la función addHarmonics por separado
def apply_harmonics(image):
    return addHarmonics(image, omega, start_date)

# Aplicar la función addHarmonics a la colección de imágenes usando map
landsatPlus = landsat.map(apply_harmonics)

#print('Landsat collection with harmonic basis: ', landsatPlus)

In [19]:
# Function to run linear regression on an image.
def arrayimgHarmonicRegr(harmonicColl, dependent, independents):
    independents = ee.List(independents)
    dependent = ee.String(dependent)

    regression = harmonicColl \
        .select(independents.add(dependent)) \
        .reduce(ee.Reducer.linearRegression(independents.length(), 1))

    return regression

# Function to extract and rename regression coefficients.
def imageHarmonicRegr(harmonicColl, dependent, independents):
    hregr = arrayimgHarmonicRegr(harmonicColl, dependent, independents)

    independents = ee.List(independents)
    dependent = ee.String(dependent)

    def func_mxk(b):
        return dependent.cat(ee.String('_')).cat(ee.String(b))

    newNames = independents.map(func_mxk)

    imgCoeffs = hregr.select('coefficients') \
        .arrayProject([0]) \
        .arrayFlatten([independents]) \
        .select(independents, newNames)

    return imgCoeffs

# Function to apply imageHarmonicRegr and create a multi-band image.
def apply_imageHarmonicRegr(band):
    return imageHarmonicRegr(harmonicColl, band, independents)

# Apply getHarmonicCoeffs to ImageCollection.
def getHarmonicCoeffs(harmonicColl, bands, independents):
    coefficients = ee.ImageCollection.fromImages(
        bands.map(apply_imageHarmonicRegr)
    )
    return coefficients.toBands()

In [20]:
# Definir harmonicColl antes de usarla, si no está definida antes en tu código
harmonicColl = landsatPlus

# Aplicar la función corregida
bands = ee.List(['NIR', 'SWIR1', 'SWIR2', 'GCVI'])  # Cambié la lista a un objeto ee.List
independents = ee.List(['constant', 'cos', 'sin', 'cos2', 'sin2'])
harmonics = getHarmonicCoeffs(harmonicColl, bands, independents)

harmonics = harmonics.clip(geometry)
harmonics = harmonics.multiply(10000).toInt32()

# Compute fitted values.
gcviHarmonicCoefficients = harmonics \
    .select([
        '3_GCVI_constant', '3_GCVI_cos',
        '3_GCVI_sin', '3_GCVI_cos2', '3_GCVI_sin2'
    ]) \
    .divide(10000)

def add_fitted_band(image):
    return image.addBands(
        image.select(independents) \
        .multiply(gcviHarmonicCoefficients) \
        .reduce('sum') \
        .rename('fitted')
    )

fittedHarmonic = landsatPlus.map(add_fitted_band)

In [28]:
def r2(harmonicoll, dependent, independents):
    
    
    dependent = ee.String(dependent)

    
    # Computing variance for R2
    totreducer = ee.Reducer.sampleVariance().combine(ee.Reducer.count(), None, True).combine(ee.Reducer.mean(), None, True)
    stats = harmonicoll.select(dependent).reduce(totreducer)
    #stats = stats.reproject(ee.Image(harmonicoll..projection()))
    variance = stats.select(dependent.cat(ee.String('_variance')))

    # Computing R2
    r2bandn = dependent.cat(ee.String('_r2'))
    r2 = ee.Image(1).updateMask(variance).subtract(rmse.pow(2).divide(variance)).select([0], [r2bandn])
    
    return r2

In [29]:
dependent = 'GCVI'
fitted_values = 'fitted'

In [30]:
r2(harmonics, dependent, independents)

UnboundLocalError: cannot access local variable 'independent' where it is not associated with a value

In [52]:
gcviHarmonicCoefficients.bandNames()

In [56]:
def calculate_metrics_for_image(img):
    dependent = 'GCVI'
    fitted_values = 'fitted'

    # Calcular la media de GCVI
    mean_gcvi = img.select(dependent).reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=img.geometry(),
        scale=30,
        maxPixels=1e9
    ).get(dependent)
    
    residuals = img.select(dependent).subtract(img.select(fitted_values))
    squared_residuals = residuals.pow(2)
    rmse = squared_residuals.reduce(ee.Reducer.mean()).sqrt().rename(dependent + '_rmse')
    
    
#     residual_sum_of_squares = residuals.pow(2).reduce(ee.Reducer.sum())


#     total_sum_of_squares = img.select(dependent).subtract(ee.Image.constant(mean_gcvi)).pow(2).reduce(ee.Reducer.sum())
    
#     r2 = ee.Image(1).subtract(residual_sum_of_squares.divide(total_sum_of_squares)).rename(dependent + '_r2')

    # Computing variance for R2

    totreducer = ee.Reducer.sampleVariance().combine(ee.Reducer.count(), None, True).combine(ee.Reducer.mean(), None, True)
    stats = img.select(dependent).reduce(totreducer)
    stats = stats.reproject(ee.Image(img.projection()))
    variance = stats.select('variance')

    # Computing R2
    r2bandn = ee.String(dependent).cat(ee.String('_r2'))
    r2 = ee.Image(1).updateMask(variance).subtract(rmse.pow(2).divide(variance)).select([0], [r2bandn])
    
    return r2.addBands(rmse)

In [57]:
metrics_images = fittedHarmonic.map(calculate_metrics_for_image)

In [ ]:
metrics_images.getInfo()

In [59]:
Map = geemap.Map(center=[40, -100], zoom=4)
Map.setCenter(long, lat, zoom=10)

Map.addLayer(metrics_images.select('GCVI_rmse'), {'min': 0, 'max': 3, 'palette': ['#d73027', '#fee08b', '#1a9850']}, 'RMSE')
Map.addLayer(metrics_images.select('GCVI_r2'), {'min': 0, 'max': 1, 'palette': ['#d73027', '#fee08b', '#1a9850']}, 'R2')

#Map.addLayer(region, {'color': 'red'}, 'Region')
Map

Map(center=[40.579804398254005, -88.81417685576481], controls=(WidgetControl(options=['position', 'transparent…